## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [23]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,65.91,53,99,4.85,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,78.96,71,1,16.71,clear sky
2,2,Punta Arenas,CL,-53.1500,-70.9167,57.31,47,75,21.85,broken clouds
3,3,Smithers,CA,54.7804,-127.1743,37.72,56,40,0.00,scattered clouds
4,4,Ponta Do Sol,PT,32.6667,-17.1000,64.06,58,40,5.10,scattered clouds


In [24]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [25]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,78.96,71,1,16.71,clear sky
7,7,Carutapera,BR,-1.1950,-46.0200,77.18,91,99,5.95,overcast clouds
8,8,Butaritari,KI,3.0707,172.7902,82.69,70,99,14.74,overcast clouds
9,9,Mahebourg,MU,-20.4081,57.7000,80.89,83,75,5.75,light intensity shower rain
12,12,Sao Filipe,CV,14.8961,-24.4956,77.32,46,70,20.80,broken clouds
15,15,Saint-Philippe,RE,-21.3585,55.7679,78.35,81,27,10.69,scattered clouds
20,20,Angoche,MZ,-16.2325,39.9086,76.23,92,98,5.28,light rain
22,22,Pacific Grove,US,36.6177,-121.9166,84.58,29,0,4.61,clear sky
24,24,Cabedelo,BR,-6.9811,-34.8339,83.07,77,38,7.81,scattered clouds
27,27,Lorengau,PG,-2.0226,147.2712,81.86,77,97,1.79,overcast clouds


In [26]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                188
City                   188
Country                188
Lat                    188
Lng                    188
Max Temp               188
Humidity               188
Cloudiness             188
Wind Speed             188
Current Description    188
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,78.96,clear sky,-23.1203,-134.9692,
7,Carutapera,BR,77.18,overcast clouds,-1.1950,-46.0200,
8,Butaritari,KI,82.69,overcast clouds,3.0707,172.7902,
9,Mahebourg,MU,80.89,light intensity shower rain,-20.4081,57.7000,
12,Sao Filipe,CV,77.32,broken clouds,14.8961,-24.4956,
15,Saint-Philippe,RE,78.35,scattered clouds,-21.3585,55.7679,
20,Angoche,MZ,76.23,light rain,-16.2325,39.9086,
22,Pacific Grove,US,84.58,clear sky,36.6177,-121.9166,
24,Cabedelo,BR,83.07,scattered clouds,-6.9811,-34.8339,
27,Lorengau,PG,81.86,overcast clouds,-2.0226,147.2712,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [31]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,78.96,clear sky,-23.1203,-134.9692,People ThankYou
7,Carutapera,BR,77.18,overcast clouds,-1.1950,-46.0200,Lidera
8,Butaritari,KI,82.69,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
9,Mahebourg,MU,80.89,light intensity shower rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
12,Sao Filipe,CV,77.32,broken clouds,14.8961,-24.4956,Hotel Xaguate
15,Saint-Philippe,RE,78.35,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
20,Angoche,MZ,76.23,light rain,-16.2325,39.9086,Guest House HESADA
22,Pacific Grove,US,84.58,clear sky,36.6177,-121.9166,Lovers Point Inn
24,Cabedelo,BR,83.07,scattered clouds,-6.9811,-34.8339,Hotel Pigalle
27,Lorengau,PG,81.86,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel


In [32]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))